# Loading Javascript modules

## Run it!

* make sure jupyter server proxy is installed
* copy httpserver.py and LoadingJavascriptModules.ipynb in the same directory of a Jupyter Lab server
* create a widgets subdirectory there and copy: main.js and the module subdirectory
* Run the notbook (Kernel -> Restart Kernel And Run All Cells...)

|     |     |
| --- | --- |
| **You should get this right hand picture** | ![](./widgets/expected.png) |

## What does it do?

* The normal behavior of Jupyter Lab notebook is to have all frontend components pre-loaded
* Some time, experimentation is one use-case, loading frontend components is necessary
* This notebook present a way to load javascript modules dynamically

## Libraries

In [1]:
from httpserver import start, stop
from http.server import SimpleHTTPRequestHandler, HTTPServer
import os
from urllib.parse import urlparse, parse_qs
import ipywidgets as widgets
import time

## Backend: the HTTP server

### Widgets python implementation

In [2]:
logsw=widgets.Textarea(layout=widgets.Layout(width='50%'))
logsw.value = ""
bag ={}
class myHTTPRequestHandler(SimpleHTTPRequestHandler):
    callback={}
    def __init__(self, *args, directory=None,bag=bag, **kwargs):
        self.bag = bag
        self.directory = os.path.join(os.getcwd(),"widgets")
        super().__init__(*args, directory=self.directory, **kwargs)
        # print(self.directory)
    def end_headers(self):
        super().end_headers()
    def do_GET(self):
        self.parsed_path = urlparse(self.path)
        self.queryparams = parse_qs(self.parsed_path.query)
        if self.path.endswith('/version'):
            self.version()
        elif self.parsed_path.path.endswith('/setvalue'):
            self.setvalue()
        else:
            super().do_GET()
    def version(self):
        ans = '{"version": "0.0"}'
        eans = ans.encode()        
        self.send_response(200)
        self.send_header("Content-type", "application/json")
        self.send_header("Content-Length",len(eans))
        self.end_headers()
        self.wfile.write(eans)
    def setvalue(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['value'][0]})
        if self.queryparams['variable'][0] in myHTTPRequestHandler.callback:
            self.callback[self.queryparams['variable'][0]](self.queryparams['value'][0])
        self.version() 
    def log_message(self, format, *args):
        global logsw
        v = format % args
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        logsw.value += current_time + " " + v +"\n"
logsw

Textarea(value='', layout=Layout(width='50%'))

In [3]:
start(handler_class=myHTTPRequestHandler, port=8085)

Starting httpd...



## Frontend

### Widget javascript implementation

This is the 'basic modules' example you can find [there](https://github.com/mdn/js-examples/tree/master/modules/basic-modules)

The main.js file is slightly changed

In [4]:
class LoadModule(object):
    """This class is loading the main.js module"""
    def _repr_javascript_(self):
        return '''debugger;
var id = document.getElementById("moduleMain")
if (id!=null){
    if ('makeIt' in window){
        // the module main.js is already loaded
        fetch(window.location.origin+'/proxy/8085/setvalue?value=1&variable=loaded')
    } else {
        console.log("makeIt not found!!!!")
    }
} else {
    var script= document.createElement('script');
    script.src = "/proxy/8085/main.js";
    script.type = "module"
    script.id = "moduleMain"
    document.head.appendChild(script);
    function check(){
       if (!('makeIt' in window)){
           setTimeout(function(){ check()}, 500);
       }
    }
    check()
    // sub modules are still not loaded (let them 2 secs)
    setTimeout(fetch, 2000, window.location.origin+'/proxy/8085/setvalue?value=1&variable=loaded' )
}
'''

In [5]:
class ExecuteModule(object):
    """This class execute the code once the modul is loaded"""
    def _repr_javascript_(self):
        return '''
        element = document.getElementById("myElement0");
        element.innerHTML = "";
        makeIt(element);
        '''

In [6]:
class DisplayModule(object):
    """This class is creating the element where the output of the will set UI elements"""
    def _repr_javascript_(self):
        return '''
        element.id="myElement0";
        // console.log(element);
        '''    

In [7]:
DisplayModule()

In [8]:
def loaded_cb(value):
    display(ExecuteModule())
myHTTPRequestHandler.callback.update({"loaded": loaded_cb})
LoadModule()

In [9]:
#stop()